# Hubbard One Approximation

### Structure of notebook

1. Parameters of the cattering region are defined.
2. Parameters of the fermi dirac function & Integration interval are defined.
3. Electron Density is self-consistently calculated
4. The magnetocurrent $\Delta I(m,V) = I(m) - I(-m)$ is calculated.
5. P value is calculated

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

### Path to modules

In [ ]:
import sys
sys.path.insert(0, '/Users/khhuisman/Documents/Jupyter_notebooks/Github_Coulomb_Paper_Collinear/Modules_Coulomb_Github')
import handy_functions_coulomb as hfc
# sys.path.insert(0, '<path_to_modules>')

# Chiral Scattering Region

In [ ]:
import ChiralChainModel_git
import Geometry_Git
Geometry_Git.plot3d_chain(a=1,c=1,M=1,N=8)

# Import NEGF methods

In [ ]:
import negf_HIA_git

In [ ]:
import Integration_HIA as Integration_method

# 1. Defining Hamiltonian, Leads

In [ ]:
Lm = 8             #number of sites 
chirality = True   #handedness of molecule 
N = Lm             # Number of sites within one winding
M = 1              # Number of windings
a = 1              # radius of helix
c = 1              # pitch of helix


epsilon = 0 # onsite energy
t = 2.4     # hopping paramter

# List of Spin-Orbit Coupling parameter used in publication
lambdalist = [(1*(10**-3))*t
#               ,(1*(10**-3))*t
             ]
# List of interaction strengths used in publication
Ulist = [ 0.5*t] 


#Gamma Matrix
kmax  = 4      # Number spin up + down sites coupled to each lead
gamma = 0.5    # coupling strength
pz    = 0.5    # magnetic polarization

T = 300 # Kelvin Temperature of leads
betaL,betaR = negf_HIA_git.func_beta(T), negf_HIA_git.func_beta(T) # 1/ElectronVolt



# 2. Fermi Energy, Energies to integrate over

In [ ]:
def func_energies_largeU(Hamiltonian0,U,npoints,delta_ef):
    
    '''
    Input:
    Hamiltonian0 = molecule without interaction (U=0)
    U = interaction strength
    npoints = number of energy points in window [emin,emax]
    Output:
    emax = upper bound of intergral
    emin = lowest eigenvalue of Hamiltonian0
    energiesreal = list of energies between [emin-1000,emax] (make sure the emax < ef +Vmax/2)
    eflist = list of fermi energies for which to calculate electron densities.
    
    More usefull for very large U, since it compensates for the shift with U/2
    '''
    
    evlist = np.add(np.linalg.eigh(Hamiltonian0)[0],U*np.array(hfc.list_halves(Hamiltonian0)))
    e_lumo = evlist[int(Hamiltonian0.shape[0]/2)-1] 
    e_homo = evlist[int(Hamiltonian0.shape[0]/2)]   
    #Fermi Energy
    hl_gap = e_lumo - e_homo
    
    
    
    #lower,upper bound for Glesser function
    emin = np.round(int(10*min(evlist))/10 - 10,2) #lower bound for integrals
    emax = np.round(int(10*max(evlist))/10 + 10,2) 
    
    energies_zero4000 = np.linspace(emin-6000,emin,8000)
   
    energiesreal_prime = np.linspace(emin,emax,npoints)
    
    energiesreal = hfc.jointwolist(energies_zero4000,energiesreal_prime)
    

    eflist = [(1 + delta_ef)*(U/2)]
    
    return emin,emax,npoints,energiesreal,eflist


def func_energies(Hamiltonian0,U,npoints):
    '''
    Input:
    Hamiltonian0 = molecule without interaction (U=0)
    U = interaction strength
    npoints = number of energy points in window [emin,emax]
    Output:
    emax = upper bound of intergral
    emin = lowest eigenvalue of Hamiltonian0
    energiesreal = list of energies between [emin-1000,emax] (make sure the emax < ef +Vmax/2)
    eflist = list of fermi energies for which to calculate electron densities.
    '''
    evlist = np.linalg.eigh(Hamiltonian0)[0]
    e_lumo = evlist[int(Hamiltonian0.shape[0]/2)-1]
    e_homo = evlist[int(Hamiltonian0.shape[0]/2)]
    #Fermi Energy
    hl_gap = e_lumo - e_homo
    
    
    
    #lower,upper bound for Glesser function
    emin = np.round(int(10*min(evlist))/10 - 10,2) #lower bound for integrals
    emax = np.round(int(10*max(evlist))/10 + 10,2)   #lower bound for integrals
    
    
    energies_zero4000 = np.linspace(emin-3000,emin,4000)
   
    energiesreal_prime = np.linspace(emin,emax,npoints)
    
    energiesreal = np.unique(hfc.jointwolist(energies_zero4000,energiesreal_prime))
    

#     eflist = [U/2 + abs(np.round( 0  + kappa*hl_gap/2,11)) for kappa in [0.25] ]

    eflist = [ U/2 ]
    
    return emin,emax,npoints,energiesreal,eflist

# 2.  Bias window

In [ ]:
Vmax = 0.1  # Maximum bias voltage [eV]
dV = 0.1 # stepsize
V_list_pos_bias,V_list_total = hfc.func_V_total(Vmax,dV)
print(len(V_list_total),V_list_pos_bias)

# 2. Energes to integrate over

Comment on the convergence: 
- For big $U$ $(U > t)$ use $\alpha \in [0,0.1]$.
- For small $U$ $(U < t)$ use $\alpha > 0.75 $.

In [ ]:
tol = 10**-5
tol_nintegrand = 10**-7 #minimum value for the integrand of <nis>

max_iteration = 400

npoints = 20000
alpha = 0.88


#### Set Coulomb interaction strength and SOC paramter

In [ ]:
U       = Ulist[0]
lambda1 = lambdalist[0]

GammaR,GammaLP,GammaLM,Hamiltonian0,hamiltonian_shape= ChiralChainModel_git.system_hamiltonian0(Lm,
                                                                                    epsilon,t, 
                                                                              lambda1,chirality,
                                                                              a,c,M,N,
                                                                                kmax,gamma,abs(pz)
                                                                            )


emin,emax,npoints,energies,eflist = func_energies(Hamiltonian0,U,npoints)

In [ ]:
# DOS plot to see that ef is in between the HOMO,LUMO level (these are the first two peaks one sees w.r.t. EF = U/2):
# Integration_method.plot_DOS([0],
#                                    [hfc.halves_list(Hamiltonian0)],
# eflist[0],U,Hamiltonian0, GammaLP,GammaR,0,False)

# 3. Electron Densities

Note: Hubbard One converges well for large U 

In [ ]:


for i in range(len(eflist)):
    
    ef = eflist[i]
    print(ef,lambda1/t,U/t)

    n_list_totalP,convglistP = Integration_method.self_consistent_trapz_PN(V_list_pos_bias,Vmax,
                                  max_iteration,
                                ef,
                                U,
                                Hamiltonian0,
                                GammaLP,GammaR, 
                                betaL, betaR,tol,
                                energies,tol_nintegrand,alpha,plot_bool=False,trackbool=True)


    n_list_totalM,convglistM = Integration_method.self_consistent_trapz_PN(V_list_pos_bias,Vmax,
                                  max_iteration,
                                ef,
                                U,
                                Hamiltonian0,
                                GammaLM,GammaR, 
                                betaL, betaR,tol,
                                energies,tol_nintegrand,alpha,plot_bool=False,trackbool=False)


    V_list_convg,nP_list_convg,nM_list_convg =  Integration_method.converged_lists(V_list_total,
                                                              n_list_totalP ,convglistP,
                                                              n_list_totalM, convglistM)

In [ ]:
n_list_total_convgM_swap = [hfc.pairwise_swap(nM_list_convg[i]) for i in range(len(nM_list_convg))]

plt.plot(V_list_convg,np.subtract(nP_list_convg,n_list_total_convgM_swap))
plt.xlabel('Bias Voltage [eV] ')
plt.ylabel('Electron Density')
plt.ticklabel_format(style="sci", scilimits=(0,0))


plt.show()

In [ ]:
Integration_method.plot_DOS(V_list_convg,nP_list_convg,
             ef,U,
             Hamiltonian0,GammaLP,GammaR,
             4,True)

# 4. Magnetocurrent

In [ ]:
import Currents_HIA_git as Current_method

In [ ]:
for i in range(len(eflist)):
    ef = eflist[i]
    IP_list = Current_method.calc_I_trapz(8000,
                    V_list_convg,ef,
                  Hamiltonian0 ,
                  GammaLP,GammaR,
                  U,nP_list_convg,
                  betaL,betaR)
    
    IM_list = Current_method.calc_I_trapz(8000,
                V_list_convg,ef,
              Hamiltonian0 ,
              GammaLM,GammaR,
              U,nM_list_convg,
              betaL,betaR)

In [ ]:
plt.title('$U/t = {}$, $\lambda /t = {}$'.format(U/t,lambda1/t))
plt.plot(V_list_convg,IP_list)
plt.plot(V_list_convg,IM_list)
plt.xlabel('Bias Voltage [eV]')
plt.ylabel('Current [eV]')
plt.ticklabel_format(style="sci", scilimits=(0,0))

plt.show()

In [ ]:
dIlist= np.subtract(IP_list,IM_list)
Vprime, PClist = Current_method.func_MR_list(IP_list,IM_list,V_list_convg)

In [ ]:
plt.title('Colinear $\Delta I(m): ef = {},U/t = {}$, $\lambda /t = {}$'.format(ef,U/t,lambda1/t))
plt.plot(V_list_convg,dIlist)
plt.xlabel('Bias Voltage [eV]')
plt.ylabel('Current [eV]')
plt.ticklabel_format(style="sci", scilimits=(0,0))

plt.show()

In [ ]:
plt.title('$E_F = U/2:$ $ U/t = {}$, $\lambda /t = {}$'.format(U/t,lambda1/t))
plt.plot(Vprime , PClist )
plt.xlabel('Bias Voltage')
plt.ylabel('$P_C$ [%]')
plt.ticklabel_format(style="sci", scilimits=(0,0))

plt.show()

# 5. P Value

In [ ]:
import Pvalue

In [ ]:
Vlist_prime,PJ_list = Pvalue.function_PvaluedI(V_list_convg,dIlist,8)

In [ ]:
plt.plot(Vlist_prime,PJ_list)
plt.xlim(0,Vmax)
plt.ylim(-1-0.1,1+0.1)
plt.xlabel('Bias Voltage [eV]')
plt.ylabel('P')
plt.show()